# Gluing and Merging

Within this notebook, we glue the individual files which each contain a single sweep (elevation level), into full volume scans, where each file represents a volume scan. We have downloaded the raw data from the [ARM Data portal](https://adc.arm.gov/discovery/#/results/datastream::gucxprecipradarS2.00), using the `gucxprecipradarS2.00` datastream.

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import os
import time
import datetime
import numpy as np
import xarray as xr

from matplotlib import pyplot as plt
from dask.distributed import Client, LocalCluster, progress

import pyart


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


## Define Helper Functions

In [2]:
def glue_fix(nfile):
    
    status = 'SUCCESS: ' + str(nfile)
    
    month = "202112"
    OUTPUT = "/gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/" + month + "_glued/"
    encodings = {'DBZ' : {'_FillValue' : -32768.0},
                 'VEL' : {'_FillValue' : -32768.0},
                 'WIDTH' : {'_FillValue' : -32768.0},
                 'ZDR' : {'_FillValue' : -32768.0},
                 'PHIDP' : {'_FillValue' : -32768.0},
                 'RHOHV' : {'_FillValue' : -32768.0},
                 'NCP' : {'_FillValue' : -32768.0},
                 'DBZhv' : {'_FillValue' : -32768.0},
                }
    # load the file
    try:
        ds = xr.open_dataset(nfile, mask_and_scale=False)
    except:
        ds = None
        status = "FALIURE: - can't open file: " + str(nfile)
    
    if ds is not None:
        # Loop over all the variables; update the FillValue and Data Type
        for var in encodings:
            # Convert all values within the DataArray to the correct Fill Value
            # NOTE: xr.where(condition, value_when_condition_is_not_met); so if every index is MVC, check for opposite
            mask = ds[var].where(ds[var] > -99800, encodings[var]['_FillValue'])
            # Append the corrected data to the variable.
            ds[var] = mask
        # Define output name
        nout = OUTPUT + nfile.split('/')[-1]
        # Save the file
        try:
            ds.to_netcdf(path=nout)
        except:
            status = "FAILURE - can't write: " + str(nout)
    
    return status

In [3]:
def radar_glue(b_radar, radar_list):
    if radar_list is not None:
        for rad in radar_list:
            b_radar = pyart.util.join_radar(b_radar, rad)
            del rad
    else:
        b_radar = None
    return b_radar

In [4]:
def volume_from_list(base_radar, vlist, base_dir):
    try:
        radars = [pyart.io.read(base_dir+sw) for sw in vlist[1::]]
    except:
        radars = None
    return radar_glue(base_radar, radars)

In [5]:
def granule(Dvolume):
    n_tilts = 8
    month = "202112"
    data_dir = "/gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/nc_files/" + month + "_nc/"
    out_dir = "/gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/" + month + "_glued/"
    if len(Dvolume) == 8:
        try:
            base_rad = pyart.io.read(data_dir+Dvolume[0])
        except:
            base_rad = None
        
        if base_rad is not None:
            out_radar = volume_from_list(base_rad, Dvolume, data_dir)
            if out_radar is not None:
                ff = time.strptime(out_radar.time['units'][14:], '%Y-%m-%dT%H:%M:%SZ')
                dt = datetime.datetime.fromtimestamp(time.mktime(ff)) + datetime.timedelta(seconds= int(out_radar.time['data'][0]))
                strform = dt.strftime(out_dir + 'xprecipradar_guc_volume_%Y%m%d-%H%M%S.b1.nc')
                #FIX for join issue.. to be fixed in Py-ART
                out_radar.sweep_mode['data']=np.tile(base_rad.sweep_mode['data'], n_tilts)
                try:
                    pyart.io.write_cfradial(strform, out_radar, arm_time_variables=True)
                    print('SUCCESS', strform)
                except:
                    print('FAILURE', strform)
                # Delete the radars to free up memory
                del base_rad
                del out_radar

## Locate all the PPI scans within the Desired Directories

In [6]:
month = "202112"
data_dir = "/gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/nc_files/" + month + "_nc/"
out_dir = "/gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/" + month + "_glued/"

In [7]:
all_files = os.listdir(data_dir)
all_files.sort()

In [8]:
base_scan_ppi = '1_PPI.nc'
ppi_pattern = 'PPI.nc'
base_scans = []
volumes = []
ppis = []
in_volume = False
for file in all_files:
    if ppi_pattern in file:
        ppis.append(file)
    if base_scan_ppi in file:
        base_scans.append(file)
    

In [9]:
n_tilts = 8

In [10]:
volumes = []
for base in base_scans:
    base_scan_index = np.where(np.array(ppis) == base)[0][0]
    #print(base_scan_index)
    volume = ppis[base_scan_index: base_scan_index+n_tilts]
    volumes.append(volume)

## Try Single Volume to Verify Process

In [11]:
volumes[0]

['gucxprecipradarS2.00.20211201.000248.raw.csu.sail-20211201-000248_40208_22_1_PPI.nc',
 'gucxprecipradarS2.00.20211201.000320.raw.csu.sail-20211201-000320_40209_22_2_PPI.nc',
 'gucxprecipradarS2.00.20211201.000352.raw.csu.sail-20211201-000352_40210_22_4_PPI.nc',
 'gucxprecipradarS2.00.20211201.000424.raw.csu.sail-20211201-000424_40211_22_6_PPI.nc',
 'gucxprecipradarS2.00.20211201.000456.raw.csu.sail-20211201-000456_40212_22_8_PPI.nc',
 'gucxprecipradarS2.00.20211201.000528.raw.csu.sail-20211201-000528_40213_22_10_PPI.nc',
 'gucxprecipradarS2.00.20211201.000600.raw.csu.sail-20211201-000600_40214_22_12_PPI.nc',
 'gucxprecipradarS2.00.20211201.000632.raw.csu.sail-20211201-000632_40215_22_15_PPI.nc']

In [12]:
%%time
granule(volumes[0])

SUCCESS /gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202112_glued/xprecipradar_guc_volume_20211201-000248.b1.nc
CPU times: user 9.7 s, sys: 1.62 s, total: 11.3 s
Wall time: 13 s


## Start a Dask Cluster

In [13]:
from dask.distributed import Client, LocalCluster

cluster = LocalCluster()

client = Client(cluster)
client

/ccsopen/home/jrobrien/.conda/envs/sail_act_pyart/lib/python3.10/site-packages/tornado/platform/asyncio.py:193: DeprecationWarning: There is no current event loop
  old_loop = asyncio.get_event_loop()
/ccsopen/home/jrobrien/.conda/envs/sail_act_pyart/lib/python3.10/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34861 instead
  warnings.warn(
2023-05-22 14:54:07,441 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-enpmlc14', purging
2023-05-22 14:54:07,441 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-9ucyg2ag', purging
2023-05-22 14:54:07,441 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-840d9c4k', purging
2023-05-22 14:54:07,441 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:34861/status,
Dashboard: http://127.0.0.1:34861/status,Workers: 16
Total threads: 128,Total memory: 251.48 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43125,Workers: 16
Dashboard: http://127.0.0.1:34861/status,Total threads: 128
Started: Just now,Total memory: 251.48 GiB
Comm: tcp://127.0.0.1:42045,Total threads: 8
Dashboard: http://127.0.0.1:33539/status,Memory: 15.72 GiB
Nanny: tcp://127.0.0.1:42247,


## Glue the Files

In [14]:
future = client.map(granule, volumes)

In [15]:
%%time
my_data = client.gather(future)

<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
<frozen importlib._bootstrap>:283: Deprecation

KilledWorker: Attempted to run task granule-4902ee5ab80d67acd599409b8a32b3c3 on 3 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:37037. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
2023-05-22 15:03:25,375 - distributed.nanny - WARNING - Restarting worker
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
2023-05-22 15:03:44,790 - distributed.nanny - WARNING - Restarting worker
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
2023-05-22 15:04:20,309 - distributed.nanny - WARNING - Restarting worker
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
2023-05-22 15:07:19,983 - distributed.nanny - WARNING - Restarting worker
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_modu


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119

SUCCESS /gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202112_glued/xprecipradar_guc_volume_20211209-201800.b1.nc
SUCCESS /gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202112_glued/xprecipradar_guc_volume_20211209-203919.b1.nc
SUCCESS /gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202112_glued/xprecipradar_guc_volume_20211209-203400.b1.nc
SUCCESS /gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202112_glued/xprecipradar_guc_volume_20211209-202319.b1.nc
SUCCESS /gpfs/wolf/atm124/proj-shared/

2023-05-22 15:12:37,533 - distributed.nanny - WARNING - Restarting worker


...

SUCCESS /gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202112_glued/xprecipradar_guc_volume_20211203-085253.b1.nc
SUCCESS /gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202112_glued/xprecipradar_guc_volume_20211208-050216.b1.nc
SUCCESS /gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202112_glued/xprecipradar_guc_volume_20211210-080722.b1.nc
SUCCESS /gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202112_glued/xprecipradar_guc_volume_20211213-012746.b1.nc
SUCCESS /gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202112_glued/xprecipradar_guc_volume_20211214-154304.b1.nc
SUCCESS /gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202112_glued/xprecipradar_guc_volume_20211212-011140.b1.nc
SUCCESS /gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202112_glued/xprecipradar_guc_volume_20211212-092222.b1.nc
SUCCESS /gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202112_glued/xprecipradar_g

2023-05-22 16:54:00,630 - distributed.nanny - WARNING - Restarting worker
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
HDF5-DIAG: Error detected in HDF5 (1.12.2) thread 1:
  #000: H5F.c line 532 in H5Fcreate(): unable to create file
    major: File accessibility
    minor: Unable to open file
  #001: H5VLcallback.c line 3282 in H5VL_file_create(): file create failed
    major: Virtual Object Layer
    minor: Unable to create file
  #002: H5VLcallback.c line 3248 in H5VL__file_create(): file create failed
    major: Virtual Object Layer
    minor: Unable to create file
  #003: H5VLnative_file.c line 63 in H5VL__native_file_create(): unable to create file
    major: File accessibility
    minor: Unable to open file
  #004: H5Fint.c line 1898 in H5F_open(): unable to lock the file
    major: File accessibility
    minor: Unable to lock file
  #005: H5FD.c line 1625 in H5FD_loc


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119

SUCCESS /gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202112_glued/xprecipradar_guc_volume_20211208-144936.b1.nc
SUCCESS /gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202112_glued/xprecipradar_guc_volume_20211228-131354.b1.nc
SUCCESS /gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202112_glued/xprecipradar_guc_volume_20211226-195344.b1.nc
SUCCESS /gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202112_glued/xprecipradar_guc_volume_20211216-002035.b1.nc
SUCCESS /gpfs/wolf/atm124/proj-shared/

HDF5-DIAG: Error detected in HDF5 (1.12.2) thread 4:
  #000: H5F.c line 532 in H5Fcreate(): unable to create file
    major: File accessibility
    minor: Unable to open file
  #001: H5VLcallback.c line 3282 in H5VL_file_create(): file create failed
    major: Virtual Object Layer
    minor: Unable to create file
  #002: H5VLcallback.c line 3248 in H5VL__file_create(): file create failed
    major: Virtual Object Layer
    minor: Unable to create file
  #003: H5VLnative_file.c line 63 in H5VL__native_file_create(): unable to create file
    major: File accessibility
    minor: Unable to open file
  #004: H5Fint.c line 1898 in H5F_open(): unable to lock the file
    major: File accessibility
    minor: Unable to lock file
  #005: H5FD.c line 1625 in H5FD_lock(): driver lock request failed
    major: Virtual File Layer
    minor: Unable to lock file
  #006: H5FDsec2.c line 1002 in H5FD__sec2_lock(): unable to lock file, errno = 11, error message = 'Resource temporarily unavailable'
    m


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119

FAILURE /gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202112_glued/xprecipradar_guc_volume_20211203-213203.b1.nc
SUCCESS /gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202112_glued/xprecipradar_guc_volume_20211229-014557.b1.nc
SUCCESS /gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202112_glued/xprecipradar_guc_volume_20211221-201913.b1.nc
SUCCESS /gpfs/wolf/atm124/proj-shared/gucxprecipradarS2.00/glue_files/202112_glued/xprecipradar_guc_volume_20211201-012231.b1.nc
SUCCESS /gpfs/wolf/atm124/proj-shared/